In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn

In [2]:
ctx = mx.gpu()

In [3]:
class AutoRec(nn.HybridBlock):
    def __init__(self, layerwise_dims, dropout_prob=0, verbose=False, **kwargs):
        super(AutoRec, self).__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            self.user_embedding = nn.Embedding(input_dim=n_users, output_dim=100)
            self.movie_embedding = nn.Embedding(input_dim=n_movies, output_dim=100)
            
        self.encoder = nn.HybridSequential()
        with self.encoder.name_scope():
            for num_units in layerwise_dims:
                self.encoder.add(nn.Dense(num_units, activation='relu'))
            self.encoder.add(nn.Dropout(dropout_prob))
            
        self.decoder = nn.HybridSequential()
        with self.decoder.name_scope():
            for ind, num_units in enumerate(list(reversed(layerwise_dims))[1:]):
                self.decoder.add(nn.Dense(num_units, activation='relu'))
        
        if self.verbose:
            print('-' * 60)
            print('Network Archs: ')
            print('\nEncoder')
            print(self.encoder)
            print('\nDecoder')
            print(self.decoder)
            print('-' * 60)
    
    def hybrid_forward(self, F, user, movie):
        # concat & flatten input for autoencoder
        concat = F.concat(user, movie)
        flat = F.flatten(concat)
        
        # forward pass
        return self.decoder(self.encoder(flat))

In [4]:
# input
n_users = 100
n_movies = 100
user = mx.nd.random.uniform(shape=(4, n_users))
movie = mx.nd.random.uniform(shape=(4, n_movies))

# layer params
layers = [1024, 512, 128, 64]

# network
autorec = AutoRec(layers, dropout_prob=.5, verbose=True)
autorec.initialize()
autorec(user, movie)

------------------------------------------------------------
Network Archs: 

Encoder
HybridSequential(
  (0): Dense(None -> 1024, Activation(relu))
  (1): Dense(None -> 512, Activation(relu))
  (2): Dense(None -> 128, Activation(relu))
  (3): Dense(None -> 64, Activation(relu))
  (4): Dropout(p = 0.5)
)

Decoder
HybridSequential(
  (0): Dense(None -> 128, Activation(relu))
  (1): Dense(None -> 512, Activation(relu))
  (2): Dense(None -> 1024, Activation(relu))
)
------------------------------------------------------------



[[ 0.          0.00169634  0.00377089 ...,  0.00121543  0.          0.00138937]
 [ 0.          0.00016627  0.00216765 ...,  0.00034967  0.          0.00186136]
 [ 0.          0.00170166  0.0015621  ...,  0.00245271  0.          0.00114533]
 [ 0.          0.          0.00282252 ...,  0.00157228  0.          0.0016281 ]]
<NDArray 4x1024 @cpu(0)>